## 사전 환경 설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=79718647743b6caeb32182fd238e229de5ed567802b486eefdd34dcf329de78c
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [3]:
# import library
import pandas as pd
import numpy as np
import cv2
import os
import PIL
import random
import glob
import time
import shutil
from tqdm import tqdm
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import EfficientNet

# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device : {device}')

# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
print(f'seed : {seed}')

# set path
os.chdir('/content/drive/MyDrive/dirty_mnist')
ROOT_PATH = os.getcwd()
print(f'ROOT_PATH : {ROOT_PATH}')

device : cuda:0
seed : 42
ROOT_PATH : /content/drive/MyDrive/dirty_mnist


## 실험의 가능 여부 확인

In [ ]:
# train data의 label 저장
y_train = pd.read_csv('data/dirty_mnist_2nd_answer.csv')

In [ ]:
# 1000개의 data와 전체 train data간의 정답 분포 확인
'''
약 2%내외의 차이를 보이며, 최대 3.5%까지 차이가 나는 것 확인
2000개의 샘플데이터로 하는 테스트가 불안정하리라 판단
'''

y_train.mean(axis=0) - y_train[:1000].mean(axis=0)

index    24500.00000
a           -0.02486
b            0.00684
c            0.00122
d           -0.01054
e           -0.02628
f            0.00864
g           -0.01128
h           -0.00136
i           -0.01962
j           -0.00132
k           -0.00844
l           -0.00206
m           -0.03528
n            0.02230
o            0.01060
p           -0.00050
q            0.03720
r           -0.02214
s           -0.00246
t            0.01220
u            0.00518
v           -0.00148
w            0.01562
x           -0.00846
y            0.01460
z            0.02344
dtype: float64

In [ ]:
# 2000개의 data와 전체 train data간의 정답 분포 확인
'''
약 1%내외의 차이를 보이며, 최대 2.1%까지 차이가 나는 것 확인
2000개의 샘플데이터로 하는 테스트가 유의미하리라 판단
'''

y_train.mean(axis=0) - y_train[:2000].mean(axis=0)

index    24000.00000
a           -0.01536
b            0.00134
c           -0.00378
d            0.00046
e           -0.02178
f           -0.00336
g           -0.01778
h           -0.00636
i           -0.01062
j            0.00168
k           -0.00294
l           -0.00406
m           -0.01728
n            0.00330
o            0.02710
p            0.00300
q            0.01570
r           -0.00464
s            0.00704
t            0.01220
u            0.00968
v            0.00052
w            0.00762
x           -0.00446
y            0.00860
z            0.01744
dtype: float64

## 실험용 데이터 분리

In [ ]:
# 실험용 데이터를 저장하기 위한 디렉토리 생성
os.mkdir(os.path.join(ROOT_PATH, 'data/demo_data'))

In [ ]:
# 전체 train data중 2000개의 데이터를 demo_data 디렉토리로 복사
for i in range(2000) : 
    shutil.copy('data/train_data/'+f'{i}'.zfill(5)+'.png', 'data/demo_data')

In [ ]:
# 전체 train data 중 2000개에 대한 label 저장
y_train[:2000].to_csv('data/demo_label.csv', index=False)

In [ ]:
# 실험용 label 확인
demo_label = pd.read_csv('data/demo_label.csv')
demo_label

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,1,1,1
1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1
2,2,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0
3,3,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0
4,4,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1,1,1,0,1,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,0
1996,1996,0,1,1,0,0,0,1,0,1,1,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,1
1997,1997,0,1,1,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,0,0,0,0,0,1,1
1998,1998,1,0,0,1,1,0,0,1,1,1,1,1,0,1,0,0,0,1,0,0,1,1,0,1,1,0


## 실험용 데이터 로드

In [ ]:
# 실험용으로 분리한 데이터 로드
df_demo_labels = pd.read_csv('data/demo_label.csv')
demo_dir = np.array(sorted(glob.glob('data/demo_data/*.png')))  # 00000.png ~ 1999.png
demo_labels = np.array(df_demo_labels.values[:,1:])  # (2000, 26)

In [ ]:
# 2000개의 이미지 저장
imgs = []
for img_file in tqdm(demo_dir) :
    img = cv2.imread(img_file, cv2.IMREAD_COLOR)
    imgs.append(img)

imgs = np.array(imgs)

In [ ]:
# 데이터 셋 정의
class MnistDataset(Dataset) :
    def __init__(self, imgs=None, labels=None, transform=None) :
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    
    def __len__(self) :
        return len(self.imgs)
    
    def __getitem__(self, idx) :
        img = self.imgs[idx]
        label = torch.FloatTensor(self.labels[idx])
        if self.transform :
            img = self.transform(img)
        return img, label

In [ ]:
# 실험용 데이터를 train과 validation으로 구분
'''
5개의 폴드로 분리하여, 1600개의 data로 train하고, 400개의 data로 validation 수행
'''
kf = KFold(n_splits=5, shuffle=True, random_state=seed)
folds = []
for train_idx, valid_idx in kf.split(range(2000)) :
    folds.append((train_idx, valid_idx))

## 실험용 모델 정의

In [ ]:
# 실험용 모델 정의
'''
EfficientNet 사용을 목적으로 한 실험이었기 때문에, 이를 back bone으로 구성
최종적으로 사용할 모델은 b7이었으나, 시간과 성능을 고려하여 b4로 실험 진행
'''
class MnistEfficientNet(nn.Module) :
    def __init__(self, in_channels) :
        super(MnistEfficientNet, self).__init__()
        self.EffNet = EfficientNet.from_pretrained('efficientnet-b4', in_channels=in_channels)
        self.FC = nn.Linear(1000, 26)
    
    def forward(self, x) :
        x = F.relu(self.EffNet(x))
        x = torch.sigmoid(self.FC(x))
        return x

## 실험용 설정

아래의 값에 따른 성능의 변화를 실험하고자 코드를 구성했다.  
각종 agmentation과 hyper parameter의 값을 조정하면서 학습을 시켰고, 학습 간 정보를 log로 저장하는 코드다.  

In [ ]:
train_transform = T.Compose([
    T.ToPILImage(),
    # T.Resize((512,512)),
    T.RandomHorizontalFlip(p=0.2),
    T.RandomVerticalFlip(p=0.2),
    T.RandomRotation(30),
    T.ToTensor()
    # ,T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

valid_transform = T.Compose([
    T.ToPILImage(),
    # T.Resize((512,512)),
    T.ToTensor()
    # ,T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
batch_size = 16
lr = 0.001
epochs = 20
lr_scheduler_step = 5
lr_scheduler_gamma = 0.75

## 실험용 학습 및 로깅

In [ ]:
# 각 실험의 설정을 log file에 기록
f = open('train_option.log', 'a')
f.write('-----Transforms-----\n')
f.write(f'train_transform : {train_transform}\n')
f.write(f'valid_transform : {valid_transform}\n')
f.write('-----Train Option-----\n')
f.write(f'batch_size : {batch_size},\tlr : {lr},\tepochs : {epochs}\n')
f.write(f'lr_scheduler_step : {lr_scheduler_step},\tlr_scheduler_gamma : {lr_scheduler_gamma}\n\n')

In [ ]:
# 실험용 학습
'''
위의 설정에 따라 학습이 진행되고, 각 epoch마다 값들이 log file에 저장됨.
'''
start_time = time.time()
for fold in range(1) :
    model = MnistEfficientNet(in_channels=3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=lr_scheduler_step, gamma=lr_scheduler_gamma)
    criterion = torch.nn.BCELoss()

    # Data
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]
    train_dataset = MnistDataset(imgs=imgs[train_idx], labels=demo_labels[train_idx], transform=train_transform)
    valid_dataset = MnistDataset(imgs=imgs[valid_idx], labels=demo_labels[valid_idx], transform=valid_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size//2, shuffle=False)

    # Train
    for epoch in range(epochs) :
        f.write(f'epoch : {epoch}\n')
        train_acc_list = []
        train_loss_list = []
        with tqdm(train_loader,
                total=train_loader.__len__(),
                unit='batch') as train_bar :
            for img, label in train_bar :
                train_bar.set_description(f'Train Epoch {epoch+1} / {epochs}')
                X = img.to(device)
                y = label.to(device)

                optimizer.zero_grad()
                model.train()
                y_probs = model(X)
                loss = criterion(y_probs, y)
                loss.backward()
                optimizer.step()

                y_probs = y_probs.cpu().detach().numpy()
                label = label.detach().numpy()
                y_preds = y_probs > 0.5
                batch_acc = (label == y_preds).mean()
                train_acc_list.append(batch_acc)
                train_acc = np.mean(train_acc_list)
                train_loss_list.append(loss.item())
                train_loss = np.mean(train_loss_list)
                train_bar.set_postfix(train_loss = train_loss,
                                      train_acc = train_acc)
        
        f.write(f'\ttrain_acc : {train_acc:.5},\ttrain_loss : {train_loss:.5}\n')
    
    # Valid
        valid_acc_list = []
        valid_loss_list = []
        with tqdm(valid_loader,
                total=valid_loader.__len__(),
                unit='batch') as valid_bar :
            for img, label in valid_bar :
                valid_bar.set_description(f'Valid Epoch {epoch+1} / {epochs}')
                X = img.to(device)
                y = label.to(device)

                optimizer.zero_grad()
                model.eval()
                y_probs = model(X)
                loss = criterion(y_probs, y)

                y_probs = y_probs.cpu().detach().numpy()
                label = label.detach().numpy()
                y_preds = y_probs > 0.5
                batch_acc = (label == y_preds).mean()
                valid_acc_list.append(batch_acc)
                valid_acc = np.mean(valid_acc_list)
                valid_loss_list.append(loss.item())
                valid_loss = np.mean(valid_loss_list)
                valid_bar.set_postfix(valid_loss = valid_loss,
                                      valid_acc = valid_acc)
        
        f.write(f'\tvalid_acc : {valid_acc:.5},\tvalid_loss : {valid_loss:.5}\n')
        lr_scheduler.step()

f.write('='*20)
f.write(f' time : {int(time.time()-start_time)}sec\n\n')
f.close()

## 실험 결과 공유

실험의 대상은 `lr`, `batch_size`, `flip`, `rotation`, `resize`, `normalize`, `option_combine`이다.  
default setting을 정한 상태에서 하나의 값만 변화시키며 성능을 확인했다.  

기준인 default setting은 아래와 같다.
```
-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor())

-----Train Option-----
batch_size : 16,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75
```

default 설정에서의 대략적인 성능은 다음과 같다.
```
epoch : 15
	train_acc : 0.70654,	train_loss : 0.56011
	valid_acc : 0.59538,	valid_loss : 0.70442
epoch : 16
	train_acc : 0.72341,	train_loss : 0.53619
	valid_acc : 0.59231,	valid_loss : 0.72589
epoch : 17
	train_acc : 0.7437,	train_loss : 0.51063
	valid_acc : 0.58846,	valid_loss : 0.73811
epoch : 18
	train_acc : 0.76135,	train_loss : 0.48662
	valid_acc : 0.59894,	valid_loss : 0.76163
epoch : 19
	train_acc : 0.77647,	train_loss : 0.46224
	valid_acc : 0.59683,	valid_loss : 0.78204
```

In [21]:
# default 설정의 총 log
'''
모든 실험은 이 설정을 기반으로 하며, 하나의 설정만 미세하게 값을 변경하며 실험
15 epoch부터 val_acc는 오르지 않아 0.6을 넘기지 못하고, train_acc는 0.75를 상회
'''

with open('./logs/train_option_default.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')

-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 16,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.52178,	train_loss : 0.69698
	valid_acc : 0.53231,	valid_loss : 0.70146
epoch : 1
	train_acc : 0.54012,	train_loss : 0.68867
	valid_acc : 0.53981,	valid_loss : 0.69019
epoch : 2
	train_acc : 0.5518,	train_loss : 0.68472
	valid_acc : 0.54423,	valid_loss : 0.69414
epoch : 3
	train_acc : 0.56084,	train_loss : 0.68142
	valid_acc : 0.51837,	valid_loss : 0.69742
epoch : 4
	train_acc : 0.56611,	train_loss : 0.67833
	valid_acc : 0.55173,	valid_loss : 0.702
epoch : 5
	train_acc : 0.58034,	train_loss : 0.67215
	valid_acc : 0.56144,	valid_loss : 0.69127
epoch : 6
	train_acc : 0.59108,	train_loss : 0.66567


In [20]:
# lr 값 변화에 따른 성능 실험
'''
0.0002부터 0.002까지 0.0002만큼 더하면서 실험
0.001 보다 낮은 경우에 더 빠르고 심하게 과대적합의 신호가 등장
0.001 보다 높은 경우에 train_acc도 향상되지 않는 과소적합 신호 등장
'''

with open('./logs/train_option_lr.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')

-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 16,	lr : 0.0002,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.51793,	train_loss : 0.69713
	valid_acc : 0.5201,	valid_loss : 0.69355
epoch : 1
	train_acc : 0.53815,	train_loss : 0.68977
	valid_acc : 0.52837,	valid_loss : 0.6917
epoch : 2
	train_acc : 0.5488,	train_loss : 0.68505
	valid_acc : 0.52827,	valid_loss : 0.69486
epoch : 3
	train_acc : 0.56139,	train_loss : 0.68049
	valid_acc : 0.53192,	valid_loss : 0.69356
epoch : 4
	train_acc : 0.57796,	train_loss : 0.67186
	valid_acc : 0.54846,	valid_loss : 0.68751
epoch : 5
	train_acc : 0.59822,	train_loss : 0.66205
	valid_acc : 0.55279,	valid_loss : 0.69174
epoch : 6
	train_acc : 0.6157,	train_loss : 0.65073


In [22]:
# batch_size 값 변화에 따른 성능 실험
'''
24, 16, 8, 4의 값으로 실험
작을수록 train_acc와 val_acc의 성능이 유사하나, 성능 향상의 속도가 느림.
클수록 train_acc와 val_acc의 간격이 벌어지며, 과대적합의 신호가 빨리 등장함.
단, 이 실험은 2000개의 데이터로 진행되었기 때문에 batch_size에 대해서는 재고가 필요
'''

with open('./logs/train_option_batch_size.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')


-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 24,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.52409,	train_loss : 0.69742
	valid_acc : 0.5263,	valid_loss : 0.71125
epoch : 1
	train_acc : 0.54057,	train_loss : 0.68893
	valid_acc : 0.5345,	valid_loss : 0.70591
epoch : 2
	train_acc : 0.54773,	train_loss : 0.68543
	valid_acc : 0.5231,	valid_loss : 0.71962
epoch : 3
	train_acc : 0.55518,	train_loss : 0.68186
	valid_acc : 0.52974,	valid_loss : 0.7338
epoch : 4
	train_acc : 0.57041,	train_loss : 0.67666
	valid_acc : 0.53248,	valid_loss : 0.70223
epoch : 5
	train_acc : 0.58429,	train_loss : 0.67066
	valid_acc : 0.56057,	valid_loss : 0.69919
epoch : 6
	train_acc : 0.59661,	train_loss : 0.66379


In [23]:
# flip 값 변화에 따른 성능 실험
'''
0.1부터 0.8까지 0.1씩 조정하며 실험
0.4부터 0.6사이에서 일반화 성능이 가장 높게 측정됨.
이 값을 벗어나는 경우 train_acc가 빠르게 높아지며, val_acc는 개선되지 않음.
'''

with open('./logs/train_option_flip.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')

-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.1)
    RandomVerticalFlip(p=0.1)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 16,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.52627,	train_loss : 0.69723
	valid_acc : 0.51875,	valid_loss : 0.78231
epoch : 1
	train_acc : 0.54224,	train_loss : 0.68778
	valid_acc : 0.53221,	valid_loss : 0.69696
epoch : 2
	train_acc : 0.55387,	train_loss : 0.68464
	valid_acc : 0.54385,	valid_loss : 0.69253
epoch : 3
	train_acc : 0.56236,	train_loss : 0.68107
	valid_acc : 0.54558,	valid_loss : 0.70042
epoch : 4
	train_acc : 0.57024,	train_loss : 0.67685
	valid_acc : 0.54606,	valid_loss : 0.7007
epoch : 5
	train_acc : 0.5845,	train_loss : 0.6693
	valid_acc : 0.56577,	valid_loss : 0.68856
epoch : 6
	train_acc : 0.59719,	train_loss : 0.6634
	

In [24]:
# rotation 값 변화에 따른 성능 실험
'''
0.1부터 0.4까지 0.05~0.1씩 조정하며 실험
0.4에서 일반화 성능이 가장 높게 측정됨.
이 값보다 작은 경우 train_acc가 빠르게 높아지며, val_acc는 개선되지 않음.
'''

with open('./logs/train_option_rotation.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')

-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-10.0, 10.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 16,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.52329,	train_loss : 0.6971
	valid_acc : 0.51962,	valid_loss : 0.72793
epoch : 1
	train_acc : 0.53827,	train_loss : 0.68922
	valid_acc : 0.53048,	valid_loss : 0.79802
epoch : 2
	train_acc : 0.55236,	train_loss : 0.68465
	valid_acc : 0.53875,	valid_loss : 0.70067
epoch : 3
	train_acc : 0.5644,	train_loss : 0.68039
	valid_acc : 0.53183,	valid_loss : 0.70422
epoch : 4
	train_acc : 0.57038,	train_loss : 0.67739
	valid_acc : 0.54471,	valid_loss : 0.69252
epoch : 5
	train_acc : 0.58587,	train_loss : 0.66881
	valid_acc : 0.56115,	valid_loss : 0.69418
epoch : 6
	train_acc : 0.59988,	train_loss : 0.66035

In [25]:
# normalize 값 변화에 따른 성능 실험
'''
normalize가 없는 경우와 있는 경우로 구분하고, 일반적으로 사용되는 두 값에 대해 실험
(mean=[0.4913, 0.4821, 0.4465], std=[0.247, 0.2434, 0.2615])에서 일반화 성능이 가장 높게 측정됨.
normalize가 없는 경우보다 일반화 성능이 확실히 높았으며, pytorch의 document에서 제공하는 값이 더 높은 성능을 보였다.
'''

with open('./logs/train_option_normalize.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')


-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 16,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.52202,	train_loss : 0.69712
	valid_acc : 0.5349,	valid_loss : 0.69874
epoch : 1
	train_acc : 0.54111,	train_loss : 0.68879
	valid_acc : 0.53798,	valid_loss : 0.69169
epoch : 2
	train_acc : 0.55793,	train_loss : 0.68421
	valid_acc : 0.53279,	valid_loss : 0.69443
epoch : 3
	train_acc : 0.55966,	train_loss : 0.68107
	valid_acc : 0.53731,	valid_loss : 0.71499
epoch : 4
	train_acc : 0.56668,	train_loss : 0.67811
	valid_acc : 0.54692,	valid_loss : 0.69512
epoch : 5
	train_acc : 0.58481,	train_loss : 0.67042
	valid_acc : 0.56731,	valid_loss : 0.68753
epoch : 6
	train_acc : 0.59361,	train_loss : 0.664

In [26]:
# resize 값 변화에 따른 성능 실험
'''
resize가 없는 경우와 있는 경우로 구분하고, (331x331)과 (480x480) 값에 대해 실험
이 실험의 경우에는 cuda memory 이슈로 batch_size를 8로 변경하여 진행함.
(512x512)는 batch_size 8에서도 cuda memory 이슈가 발생함.
resize가 없는 경우보다 일반화 성능이 확실히 높았으며, resize가 클 수록 높은 성능을 발휘 할 것으로 판단됨.
단, 학습 시간이 매우 길어지며, batch_size가 작아짐에 따른 성능의 변동을 유의해야 함.
'''

with open('./logs/train_option_resize.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')

-----Transforms-----
train_transform : Compose(
    ToPILImage()
    Resize(size=(331, 331), interpolation=PIL.Image.BILINEAR)
    RandomHorizontalFlip(p=0.2)
    RandomVerticalFlip(p=0.2)
    RandomRotation(degrees=[-30.0, 30.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToPILImage()
    Resize(size=(331, 331), interpolation=PIL.Image.BILINEAR)
    ToTensor()
)
-----Train Option-----
batch_size : 8,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.5232,	train_loss : 0.69606
	valid_acc : 0.5325,	valid_loss : 0.69392
epoch : 1
	train_acc : 0.53832,	train_loss : 0.68973
	valid_acc : 0.50567,	valid_loss : 0.70196
epoch : 2
	train_acc : 0.54594,	train_loss : 0.68685
	valid_acc : 0.5476,	valid_loss : 0.68686
epoch : 3
	train_acc : 0.55147,	train_loss : 0.68498
	valid_acc : 0.54865,	valid_loss : 0.6843
epoch : 4
	train_acc : 0.55267,	train_loss : 0.68387
	valid_acc : 0.55375,	valid_loss : 0.68502
epoch : 5


In [27]:
# option combine 값 변화에 따른 성능 실험
'''
위에서 도출된 좋은 option을 결합하여 어떤 시너지를 내는지 실험
이 실험을 통해 얻어진 option을 실제 학습 코드에서 사용함.
'''

with open('./logs/train_option_combine.log', 'r') as f :
    while True :
        line = f.readline()
        if not line : break
        print(line, end='')

-----Transforms-----
train_transform : Compose(
    ToPILImage()
    RandomHorizontalFlip(p=0.6)
    RandomVerticalFlip(p=0.6)
    RandomRotation(degrees=[-40.0, 40.0], resample=False, expand=False)
    ToTensor()
)
valid_transform : Compose(
    ToTensor()
)
-----Train Option-----
batch_size : 16,	lr : 0.001,	epochs : 20
lr_scheduler_step : 5,	lr_scheduler_gamma : 0.75

epoch : 0
	train_acc : 0.52267,	train_loss : 0.6973
	valid_acc : 0.51115,	valid_loss : 0.73731
epoch : 1
	train_acc : 0.53394,	train_loss : 0.69021
	valid_acc : 0.51808,	valid_loss : 0.69434
epoch : 2
	train_acc : 0.54332,	train_loss : 0.68651
	valid_acc : 0.53404,	valid_loss : 0.69448
epoch : 3
	train_acc : 0.55062,	train_loss : 0.68571
	valid_acc : 0.54106,	valid_loss : 0.7015
epoch : 4
	train_acc : 0.55596,	train_loss : 0.68291
	valid_acc : 0.5325,	valid_loss : 0.7067
epoch : 5
	train_acc : 0.56498,	train_loss : 0.67971
	valid_acc : 0.55048,	valid_loss : 0.68688
epoch : 6
	train_acc : 0.56863,	train_loss : 0.67632
	